In [151]:
__author__ = "Dongjie Fan"
import pandas as pd
import numpy as np
from shapely.geometry import Point, MultiPoint
import yaml

In [2]:
tw = pd.read_csv("./tw/tw_without_duplication.csv")
tw['Tweeted_At'] = pd.to_datetime(tw['Tweeted_At'])

In [51]:
def fetch_hour(dt):
    return dt.map(lambda x: x.hour)

tw['Hour'] = fetch_hour(tw['Tweeted_At'])

In [82]:
print tw.shape[0]
print len(tw['Username'].unique())


473629
69097


### tw_home 

In [193]:
hour4home = range(22,24,1) + range(0,3,1) + range(6,9,1)
print hour4home

[22, 23, 0, 1, 2, 6, 7, 8]


In [194]:
tw_home = tw.copy()
tw_home = tw_home[tw_home['Hour'].isin(hour4home)]
tw_home.reset_index(drop=True, inplace=True)
print "# of Tweets: {}\n% of Tweets: {:.2f}%".format(tw_home.shape[0], 100.0 * tw_home.shape[0]/tw.shape[0])
print "# of Users: {}\n% of Users: {:.2f}%".format(len(tw_home['Username'].unique()), \
                100.0 * len(tw_home['Username'].unique())/len(tw['Username'].unique()))

# of Tweets: 188422
% of Tweets: 39.78%
# of Users: 42935
% of Users: 62.14%


In [195]:
users = tw_home['Username'].unique()

In [196]:
tw_home['latlon'] = tw_home['Geo'].map(lambda x: yaml.load(x)["u'coordinates'"])

In [197]:
tw_home['geometry'] = tw_home['latlon'].map(lambda x: Point(x[0], x[1]))

In [199]:
userLoc = {}
for user in users:
    userLoc[user] = list(tw_home[tw_home['Username'] == user]['geometry'])

userCentroid = {}        #  centroid of All points per user
userCount = {}           #  the number of tweets per user
for user, points in userLoc.iteritems():
    userCentroid[user] = MultiPoint(points).centroid
    userCount[user] = len(points)

In [200]:
userCount_df = pd.DataFrame.from_dict(userCount, orient='index')
userCount_df.rename(columns={0: "Count"}, inplace=True)
userCount_df.head(4)

,Count
lancewhite,1
weathergavi,1
rashelleo,2
julietterichey,4


In [201]:
userCentroid_df = pd.DataFrame.from_dict(userCentroid, orient='index')
userCentroid_df.rename(columns={0: "Centroid"}, inplace=True)
userCentroid_df.head(4)

,Centroid
lancewhite,POINT (40.76844627 -73.86230676)
weathergavi,POINT (40.6503364 -73.79200688)
rashelleo,POINT (40.71990755 -73.79631974999999)
julietterichey,POINT (40.683006305 -73.97345427249999)


In [202]:
userInfo = userCentroid_df.join(userCount_df).sort_values(ascending=False, by='Count')
userInfo.head(5)

,Centroid,Count
MrNaassir,POINT (40.69505781493996 -73.84190734701551),583
cydiadono,POINT (40.65274092851706 -73.92940767661602),526
NewYorkCP,POINT (40.71669794999983 -74.00001601999979),487
MJStarLover,POINT (40.67316508201836 -73.93811013061925),436
mufffymendes,POINT (40.78126767757577 -73.91863279664341),429


In [205]:
userInfo.to_pickle("userInfo")